In [1]:
import os
from keras.applications import VGG16
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model
from tqdm import tqdm
import cv2
import skimage
from keras import models, layers, optimizers
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
import keras
import sklearn
from sklearn.metrics import confusion_matrix
import itertools
from keras.applications.imagenet_utils import preprocess_input

%matplotlib inline

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
imageSize=224
train_dir = "OCT2017 /train/"
test_dir =  "OCT2017 /test/"

def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'): # ignores the .DS_Store // ['DME', 'CNV', 'NORMAL', '.DS_Store', 'DRUSEN']
            if folderName in ['NORMAL']:
                label = 0
            elif folderName in ['CNV']:
                label = 1
            elif folderName in ['DME']:
                label = 2
            elif folderName in ['DRUSEN']:
                label = 3
            else:
                label = 4
            for i,image_filename in enumerate(tqdm(os.listdir(folder + folderName))):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                if i>5000:#6000:
                    break
                    
    X = np.asarray(X)
    y = np.asarray(y)

    return X,y


Loads the testing set

In [3]:
X_test, y_test= get_data(test_dir)
y_testHot = to_categorical(y_test, num_classes = 4)

  0%|          | 0/243 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
100%|██████████| 243/243 [00:03<00:00, 73.97it/s]


Loads the training set (option to make train = test for small playing)

In [4]:
X_train, y_train= get_data(train_dir)
y_trainHot = to_categorical(y_train, num_classes = 4)

# X_train = X_test ###################
# y_train = y_test ###############
# y_trainHot = y_testHot ######################

class_weight1 = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

  0%|          | 0/11348 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
  5%|▌         | 1916/37205 [00:34<10:28, 56.11it/s]


 10%|█         | 3896/37205 [01:08<09:48, 56.59it/s]


 13%|█▎        | 5000/37205 [01:28<09:31, 56.36it/s]

  0%|          | 0/26315 [00:00<?, ?it/s]

  0%|          | 7/26315 [00:00<06:46, 64.70it/s]

  0%|          | 12/26315 [00:00<08:06, 54.04it/s]

  0%|          | 18/26315 [00:00<08:13, 53.29it/s]

  0%|          | 23/26315 [00:00<08:25, 52.02it/s]

  0%|          | 29/26315 [00:00<08:18, 52.71it/s]

  0%|          | 36/26315 [00:00<08:08, 53.79it/s]

  0%|          | 44/26315 [00:00<07:44, 56.55it/s]

  0%|          | 52/26315 [00:00<07:27, 58.70it/s]

  0%|          | 60/26315 [00:00<07:16, 60.14it/s]

  0%|          | 68/26315 [00:01<07:06, 61.50it/s]

  0%|          | 75/26315 [00:01<07:16, 60.09it/s]

  0%|          | 82/26315 [00:01<07:32, 57.93it/s]

  0%|          | 89/26315 [00:01<07:26, 58.72it/s]

  0%|          | 96/26315 [00:01<07:23, 59.07it/s]

  0%|          | 104/26315 [00:01<07:22, 59.27it/s]

  0%|          | 111/26315 [00:01<07:24, 58.90it/s]

  0%|          | 117/26315 [00:02<07:27, 58.49it/s]

  0%|          | 

  3%|▎         | 872/26315 [00:14<06:52, 61.62it/s]

  3%|▎         | 879/26315 [00:14<06:53, 61.53it/s]

  3%|▎         | 886/26315 [00:14<06:53, 61.42it/s]

  3%|▎         | 894/26315 [00:14<06:53, 61.54it/s]

  3%|▎         | 901/26315 [00:14<06:53, 61.53it/s]

  3%|▎         | 908/26315 [00:14<06:53, 61.51it/s]

  3%|▎         | 915/26315 [00:14<06:53, 61.42it/s]

  4%|▎         | 922/26315 [00:14<06:53, 61.47it/s]

  4%|▎         | 929/26315 [00:15<06:53, 61.44it/s]

  4%|▎         | 936/26315 [00:15<06:52, 61.47it/s]

  4%|▎         | 943/26315 [00:15<06:52, 61.47it/s]

  4%|▎         | 950/26315 [00:15<06:52, 61.45it/s]

  4%|▎         | 957/26315 [00:15<06:52, 61.44it/s]

  4%|▎         | 964/26315 [00:15<06:52, 61.45it/s]

  4%|▎         | 972/26315 [00:15<06:51, 61.55it/s]

  4%|▎         | 979/26315 [00:15<06:51, 61.52it/s]

  4%|▎         | 986/26315 [00:16<06:51, 61.50it/s]

  4%|▍         | 993/26315 [00:16<06:51, 61.51it/s]

  4%|▍         | 1000/26315 [00:16<06:52, 61.4

 11%|█         | 2860/26315 [00:45<06:14, 62.59it/s]

 11%|█         | 2868/26315 [00:45<06:14, 62.61it/s]

 11%|█         | 2876/26315 [00:45<06:14, 62.64it/s]

 11%|█         | 2884/26315 [00:46<06:14, 62.63it/s]

 11%|█         | 2891/26315 [00:46<06:13, 62.64it/s]

 11%|█         | 2899/26315 [00:46<06:13, 62.66it/s]

 11%|█         | 2906/26315 [00:46<06:13, 62.60it/s]

 11%|█         | 2913/26315 [00:46<06:13, 62.60it/s]

 11%|█         | 2920/26315 [00:46<06:14, 62.54it/s]

 11%|█         | 2928/26315 [00:46<06:13, 62.57it/s]

 11%|█         | 2935/26315 [00:46<06:13, 62.57it/s]

 11%|█         | 2942/26315 [00:47<06:13, 62.58it/s]

 11%|█         | 2949/26315 [00:47<06:13, 62.57it/s]

 11%|█         | 2956/26315 [00:47<06:13, 62.54it/s]

 11%|█▏        | 2963/26315 [00:47<06:13, 62.56it/s]

 11%|█▏        | 2970/26315 [00:47<06:13, 62.52it/s]

 11%|█▏        | 2976/26315 [00:47<06:13, 62.51it/s]

 11%|█▏        | 2982/26315 [00:47<06:13, 62.46it/s]

 11%|█▏        | 2989/26315 

 18%|█▊        | 4800/26315 [01:17<05:47, 61.85it/s]

 18%|█▊        | 4808/26315 [01:17<05:47, 61.86it/s]

 18%|█▊        | 4815/26315 [01:17<05:47, 61.86it/s]

 18%|█▊        | 4822/26315 [01:17<05:47, 61.85it/s]

 18%|█▊        | 4829/26315 [01:18<05:47, 61.83it/s]

 18%|█▊        | 4836/26315 [01:18<05:47, 61.84it/s]

 18%|█▊        | 4843/26315 [01:18<05:47, 61.85it/s]

 18%|█▊        | 4850/26315 [01:18<05:47, 61.84it/s]

 18%|█▊        | 4857/26315 [01:18<05:47, 61.83it/s]

 18%|█▊        | 4864/26315 [01:18<05:46, 61.82it/s]

 19%|█▊        | 4870/26315 [01:18<05:46, 61.81it/s]

 19%|█▊        | 4876/26315 [01:18<05:46, 61.80it/s]

 19%|█▊        | 4883/26315 [01:19<05:46, 61.81it/s]

 19%|█▊        | 4891/26315 [01:19<05:46, 61.82it/s]

 19%|█▊        | 4899/26315 [01:19<05:46, 61.84it/s]

 19%|█▊        | 4906/26315 [01:19<05:46, 61.82it/s]

 19%|█▊        | 4914/26315 [01:19<05:46, 61.83it/s]

 19%|█▊        | 4922/26315 [01:19<05:45, 61.85it/s]

 19%|█▊        | 4930/26315 

 19%|█▉        | 1644/8616 [00:27<01:56, 60.00it/s]


 19%|█▉        | 1651/8616 [00:27<01:56, 60.02it/s]


 19%|█▉        | 1658/8616 [00:27<01:55, 60.06it/s]


 19%|█▉        | 1665/8616 [00:27<01:55, 60.03it/s]


 19%|█▉        | 1672/8616 [00:27<01:55, 60.01it/s]


 19%|█▉        | 1678/8616 [00:27<01:55, 59.95it/s]


 20%|█▉        | 1684/8616 [00:28<01:55, 59.91it/s]


 20%|█▉        | 1692/8616 [00:28<01:55, 59.96it/s]


 20%|█▉        | 1699/8616 [00:28<01:55, 59.93it/s]


 20%|█▉        | 1706/8616 [00:28<01:55, 59.96it/s]


 20%|█▉        | 1713/8616 [00:28<01:55, 59.88it/s]


 20%|█▉        | 1720/8616 [00:28<01:55, 59.91it/s]


 20%|██        | 1727/8616 [00:28<01:54, 59.93it/s]


 20%|██        | 1735/8616 [00:28<01:54, 59.99it/s]


 20%|██        | 1742/8616 [00:29<01:54, 59.96it/s]


 20%|██        | 1749/8616 [00:29<01:54, 59.87it/s]


 20%|██        | 1757/8616 [00:29<01:54, 59.92it/s]


 20%|██        | 1765/8616 [00:29<01:54, 59.97it/s]


 21%|██        | 1772/8616 [

 40%|████      | 3487/8616 [00:58<01:25, 59.89it/s]


 41%|████      | 3495/8616 [00:58<01:25, 59.91it/s]


 41%|████      | 3502/8616 [00:58<01:25, 59.90it/s]


 41%|████      | 3510/8616 [00:58<01:25, 59.92it/s]


 41%|████      | 3517/8616 [00:58<01:25, 59.93it/s]


 41%|████      | 3524/8616 [00:58<01:24, 59.94it/s]


 41%|████      | 3531/8616 [00:58<01:24, 59.96it/s]


 41%|████      | 3538/8616 [00:59<01:24, 59.93it/s]


 41%|████      | 3546/8616 [00:59<01:24, 59.93it/s]


 41%|████      | 3553/8616 [00:59<01:24, 59.94it/s]


 41%|████▏     | 3560/8616 [00:59<01:24, 59.92it/s]


 41%|████▏     | 3566/8616 [00:59<01:24, 59.90it/s]


 41%|████▏     | 3574/8616 [00:59<01:24, 59.92it/s]


 42%|████▏     | 3581/8616 [00:59<01:24, 59.93it/s]


 42%|████▏     | 3588/8616 [00:59<01:23, 59.94it/s]


 42%|████▏     | 3596/8616 [00:59<01:23, 59.97it/s]


 42%|████▏     | 3603/8616 [01:00<01:23, 59.95it/s]


 42%|████▏     | 3610/8616 [01:00<01:23, 59.96it/s]


 42%|████▏     | 3617/8616 [

In [5]:
map_characters1 = {0: 'Normal', 1: 'CNV', 2: 'DME', 3: 'DRUSEN'}

In [12]:
# load VGG16
#Get back the convolutional part of a VGG network trained on ImageNet
model1 = VGG16(weights='imagenet', include_top=True, input_shape=(imageSize, imageSize, 3))
model_fc1 = Model(input=model1.input, output=model1.get_layer('block5_pool').output) # gets the output in fc1

optimizer1 = optimizers.Adam()

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  after removing the cwd from sys.path.


In [13]:
model_fc1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [14]:
class_weight1

array([1., 1., 1., 1.])

In [15]:
# Helper Functions  Learning Curves and Confusion Matrix

from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)
        
def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['acc'] # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')
    plt.ylabel('Accuracy')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')

In [16]:
def pretrainedNetwork(xtrain,ytrain,xtest,ytest,pretrainedmodel,classweight,numclasses,numepochs,optimizer,labels):
    base_model = model_fc1 # Topless

    # Add top layer
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(0.5)(x)

#### Playing with other architectures
#     x = Flatten() (x)
#     x = Dense(64) (x)
#     x = Activation('relu') (x)
#     x = Dropout(0.5) (x)
#     x = Dense(32) (x)
#     x = Activation('relu') (x)
#     x = Dropout(0.5) (x)
####

    predictions = Dense(numclasses, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Train top layer
    for layer in base_model.layers:
        layer.trainable = False
    for layer in base_model.layers[-2:]:
        layer.trainable = True
  
    model.compile(loss='categorical_crossentropy', 
          optimizer=optimizer, 
          metrics=['accuracy'])

    callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
        
    model.summary()

    # Fit model
    history = model.fit(xtrain,ytrain, epochs=numepochs, class_weight=classweight, validation_data=(xtest,ytest), verbose=1,callbacks = [MetricsCheckpoint('logs')])
    print(history)
    # Evaluate model
    score = model.evaluate(xtest,ytest, verbose=0)
    print('\nKeras CNN - accuracy:', score[1], '\n')

    y_pred = model.predict(xtest)
    Y_pred_classes = np.argmax(y_pred,axis = 1)  # reconverts back from one hot encoded 
    Y_true = np.argmax(ytest,axis = 1)  # reconverts back from one hot encoded
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
    plotKerasLearningCurve()
    plot_learning_curve(history)
    plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
    return model

numepochs = 20
pretrainedNetwork(X_train, y_trainHot, 
                 X_test, y_testHot,model1,
                 class_weight1,
                 4,numepochs,optimizer1,map_characters1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

KeyboardInterrupt: 